In [1]:
import pandas as pd
import nba_api
import bar_chart_race as bcr
import warnings
import json
warnings.simplefilter('ignore')

In [2]:
from nba_api.stats.endpoints import playergamelogs

response = playergamelogs.PlayerGameLogs(season_nullable='2021-22')
content = json.loads(response.get_json())
results = content['resultSets'][0]
headers = results['headers']
rows = results['rowSet']
df = pd.DataFrame(rows,columns= headers)

In [3]:
data = df.copy()

In [4]:
data['Date'] = pd.to_datetime(data['GAME_DATE']) - pd.to_timedelta(7, unit='d')
data = data.groupby(['PLAYER_NAME', pd.Grouper(key='Date', freq='W-MON')])['PTS'].sum().reset_index().sort_values('Date')

In [5]:
data.sort_values(by='Date',ascending=False)

,PLAYER_NAME,Date,PTS
5918,Kris Dunn,2022-04-04,9
6989,Moritz Wagner,2022-04-04,46
7596,Patrick Beverley,2022-04-04,13
1422,Chuma Okeke,2022-04-04,48
8158,Rui Hachimura,2022-04-04,72
...,...,...,...
4516,Jayson Tatum,2021-10-18,110
4491,Jaylen Nowell,2021-10-18,2
4463,Jaylen Brown,2021-10-18,85
4452,Jay Scrubb,2021-10-18,6


In [6]:
df_weekly = data.pivot_table(values = 'PTS',index = ['Date'], columns = 'PLAYER_NAME')

In [7]:
df_weekly.fillna(0, inplace=True)

In [8]:
df_weekly.iloc[:, 0:-1] = df_weekly.iloc[:, 0:-1].cumsum()

In [9]:
top_scorer = set()
for index, row in df_weekly.iterrows():
    top_scorer |= set(row[row > 0].sort_values(ascending=False).head(10).index)
df_weekly = df_weekly[top_scorer]
df_weekly.index = df_weekly.index.astype(str)
df_weekly.to_csv('NBA_Points_Bar_Plot.csv')

In [10]:
df_weekly

PLAYER_NAME,Miles Bridges,LeBron James,Joel Embiid,Ja Morant,Devin Booker,Jayson Tatum,Giannis Antetokounmpo,Zach LaVine,Trae Young,Malcolm Brogdon,...,Stephen Curry,Anthony Davis,Karl-Anthony Towns,Bradley Beal,Nikola Jokic,DeMar DeRozan,Domantas Sabonis,Luka Doncic,Paul George,Kevin Durant
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-18,100.0,78.0,63.0,105.0,55.0,110.0,98.0,102.0,75.0,99.0,...,93.0,77.0,87.0,42.0,83.0,90.0,99.0,45.0,84.0,124.0
2021-10-25,185.0,119.0,126.0,198.0,113.0,180.0,191.0,179.0,160.0,117.0,...,172.0,173.0,149.0,146.0,167.0,179.0,170.0,135.0,170.0,194.0
2021-11-01,267.0,149.0,193.0,260.0,214.0,236.0,273.0,262.0,261.0,134.0,...,276.0,263.0,212.0,218.0,254.0,269.0,212.0,249.0,243.0,324.0
2021-11-08,323.0,149.0,193.0,362.0,298.0,329.0,345.0,363.0,383.0,224.0,...,365.0,363.0,301.0,233.0,339.0,377.0,277.0,324.0,344.0,415.0
2021-11-15,395.0,182.0,193.0,433.0,381.0,463.0,457.0,467.0,450.0,292.0,...,454.0,442.0,397.0,326.0,369.0,474.0,341.0,324.0,449.0,484.0
2021-11-22,455.0,284.0,251.0,458.0,478.0,510.0,540.0,557.0,545.0,372.0,...,544.0,509.0,493.0,413.0,393.0,570.0,398.0,408.0,544.0,544.0
2021-11-29,525.0,307.0,335.0,458.0,488.0,604.0,607.0,647.0,657.0,430.0,...,637.0,561.0,558.0,494.0,460.0,633.0,464.0,457.0,599.0,629.0
2021-12-06,575.0,420.0,412.0,458.0,488.0,733.0,703.0,703.0,729.0,481.0,...,703.0,600.0,624.0,559.0,584.0,633.0,539.0,538.0,599.0,735.0
2021-12-13,655.0,493.0,502.0,474.0,504.0,802.0,703.0,703.0,791.0,493.0,...,785.0,629.0,708.0,652.0,631.0,697.0,567.0,538.0,624.0,803.0


In [11]:
bcr.bar_chart_race(df=df_weekly, 
                   n_bars=15, 
                   sort='desc',
                   title='NBA POINTS LEADERS BY WEEK',
                   period_length=1000,
                   interpolate_period=False,
                   steps_per_period=10,
                   fixed_max=True)